# Lensing EDR — Validación Numérica
Este notebook implementa el cálculo del ángulo de deflexión gravitacional en el marco EDR.
Incluye:
- Perfil de masa equivalente M_eff(r)
- Corrección δΦ_EDR
- Ángulo total de deflexión α(b)
- Radio de Einstein θ_E
- Comparación GR vs EDR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.optimize import minimize_scalar

## 1) Masa visible: perfil exponencial de disco o esferoidal

In [ ]:
def rho_visible(r, rho0=1e8, r0=5.0):
    return rho0 * np.exp(-r/r0)

def M_visible(r):
    rs = np.linspace(1e-3, r, 2000)
    integrand = 4*np.pi*rs**2 * rho_visible(rs)
    return simps(integrand, rs)

## 2) Corrección de masa EDR: M_EDR(r)
Modelo efectivo:
\begin{equation}
M_{\rm EDR}(r) = 4\pi \int_0^r r'^2 \rho_{\rm flow}(r') \, dr'
\end{equation}
con:
\begin{equation}
\rho_{\rm flow}(r) = k_{flow} \eta \Omega_{flow}^2(r)
\end{equation}

In [ ]:
def Omega_flow(r, Omega0=0.1, R_Omega=20):
    return Omega0 * np.exp(-r/R_Omega)

def rho_flow(r, kflow=0.01, eta=0.02, Omega0=0.1, R_Omega=20):
    return kflow * eta * Omega_flow(r, Omega0, R_Omega)**2

def M_EDR(r, kflow=0.01, eta=0.02, Omega0=0.1, R_Omega=20):
    rs = np.linspace(1e-3, r, 2000)
    integrand = 4*np.pi*rs**2 * rho_flow(rs, kflow, eta, Omega0, R_Omega)
    return simps(integrand, rs)

## 3) Masa efectiva total M_eff(r)

In [ ]:
def M_eff(r, kflow, eta, Omega0, R_Omega):
    return M_visible(r) + M_EDR(r, kflow, eta, Omega0, R_Omega)

## 4) Ángulo de deflexión
Se implementa:
\begin{equation}
\alpha(b) = 4G \int_b^{\infty} \frac{M_{\rm eff}(r)}{r^2 \sqrt{r^2 - b^2}} \, dr
\end{equation}

In [ ]:
def alpha_EDR(b, kflow, eta, Omega0, R_Omega):
    G = 4.30091e-6
    r = np.linspace(b*1.0001, 300, 5000)
    integrand = M_eff(r, kflow, eta, Omega0, R_Omega)/(r**2*np.sqrt(r**2 - b**2))
    return 4*G * simps(integrand, r)

## 5) Plot de comparación GR vs EDR

In [ ]:
b = np.linspace(1, 50, 200)
alpha_gr = np.array([alpha_EDR(bb, 0, 0, 0, 1) for bb in b])
alpha_edr = np.array([alpha_EDR(bb, 0.01, 0.02, 0.1, 20) for bb in b])

plt.plot(b, alpha_gr, label='GR')
plt.plot(b, alpha_edr, label='EDR')
plt.legend(); plt.xlabel('Impact parameter b'); plt.ylabel('α(b) [rad]');
plt.title('Deflexión GR vs EDR');
plt.show()